In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import mplfinance as mpf

# !!!
# Установка TA-lib под разную OS отличается, гугли как сделать make/nmake прежде чем pip install TA-lib

In [ ]:
script_path = os.path.abspath("__file__") # Определяет путь до скрипта               
directory = os.path.dirname(script_path) # Определяет путь до папки проекта
csv_folder = os.path.join(directory, 'data')
abs_paths_csv_files_list = [os.path.join(csv_folder, file) for file in os.listdir(csv_folder)]

In [ ]:
#############################################################  DATA LOADING  #############################################################
df = pd.read_csv(abs_paths_csv_files_list[0])

# convert time to datetime format
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
df['Time_alert'] = pd.to_datetime(df['Time_alert'], unit='ms')
alerting = df[df['Time'] == df['Time_alert']]

In [ ]:
############################################################## DROP COLUMNS ########################################################
def drop_columns(df):
    df = df.drop(['Quote asset volume', 'Number of trades', 'Price_alert', 'Close time', 'Time_alert', 'Taker buy base asset volume'], axis=1)
    df.set_index('Time', inplace=True)
    return df

df = drop_columns(df)
alerting = drop_columns(alerting)

alert_date = alerting.iloc[0].name
alert_date

In [ ]:
############################################################### SUPPORT LEVELS #############################################################
# Здесь группирую по частоте повторений закрытых позиций и нахожу точки в которых чаще всего останавливается график
df = df.groupby('Close').filter(lambda x: len(x) >= 1)
value_counts = df['Close'].value_counts()
total_counts = value_counts.sum()
df['Frequency'] = df['Close'].map(value_counts)
df['Importance'] = (df['Frequency'] / total_counts * 100).round(2)

In [ ]:
# Определение уровней поддержки
unique_frequencies = df['Frequency'].unique()
strong_support = [unique_frequencies.max()]
low_support = [unique_frequencies.min()]
other_supports = unique_frequencies[(unique_frequencies != strong_support) & (unique_frequencies != low_support)]

In [ ]:
delta_threshold = (df['Close'].max() - df['Close'].min())/20  # 5% 
delta_threshold

In [ ]:
import ta_indicators
# Добавление в датафрейм индикаторных признаков 
df = ta_indicators.add_features(df)

# добавление MA_50
def calculate_moving_average(data, window):
    return data.rolling(window=window).mean()

# calculate moving averages
df['MA_50'] = calculate_moving_average(df['Close'], 50)


In [ ]:
df.dropna(inplace=True)

In [ ]:
df['apo_macd'] = 0
df.loc[df['apo'] < df['macd_hist'], 'apo_macd'] = 1

df['ema_MA_50'] = 0
df.loc[df['ema'] < df['MA_50'], 'ema_MA_50'] = 1

In [ ]:
import plot_tools
colors_list_h, linewidths_list_h, hlines = plot_tools.supports_params(df, strong_support, other_supports)
hlines # уровни поддержки

In [ ]:
i = 0
while i < len(hlines) - 1:
    if abs(hlines[i] - hlines[i+1]) < delta_threshold:
        average = (hlines[i] + hlines[i+1]) / 2
        del hlines[i:i+2]
        hlines.insert(i, average)
    else:
        i += 1
hlines # уровни поддержки прореженные

In [ ]:
def dist_to_support_level(close_value, support_levels):

    if len(hlines) > 1:
        support_levels.sort()

        for i in range(len(support_levels) - 1):
            delta1 = abs(support_levels[i] - close_value)
            delta2 = abs(support_levels[i + 1] - close_value)

            if delta1 < delta2:
                return support_levels[i], delta1
            else:
                return support_levels[i + 1], delta2
            
    else:
        delta = support_levels[0] - row['Close']
        return support_levels[0], delta

    return None, None


df['nearest_support'] = None
df['delta'] = None

for index, row in df.iterrows():
    nearest_level, delta = dist_to_support_level(row['Close'], hlines)
    df.at[index, 'nearest_support'] = nearest_level
    df.at[index, 'delta'] = delta


In [ ]:
# Нахождение уровней поддержки / сопротивления по условиям
mask = (df['Close'] > df['nearest_support']) & (df['Close'].shift(1) <= df['nearest_support']) & (df['delta'].shift(5) > delta_threshold)
resistance_break = df[mask]
resistance_break = list(resistance_break.index)

mask = (df['Close'] < df['nearest_support']) & (df['Close'].shift(1) > df['nearest_support'])# & (df['Close'].shift(-10) < df['nearest_support'])
support_break = df[mask]
support_break = list(support_break.index)

all_timestamps = list(df.index)

In [ ]:
# Фильтрация избыточных уровней

from datetime import datetime, timedelta
delta = timedelta(minutes=20)

all_timestamps = list(df.index)

first_timestamp = df.iloc[0].name
last_timestamp = df.iloc[-1].name

filtered_resist = []
filtered_support = []

current_timestamp = first_timestamp
while current_timestamp <= last_timestamp:
    start_timestamp = current_timestamp
    end_timestamp = start_timestamp + delta

    current_interval = [timestamp for timestamp in all_timestamps if start_timestamp <= timestamp <= end_timestamp]

    min_timestamp = None
    max_timestamp = None
    for timestamp in current_interval:
        if timestamp in resistance_break:
            if min_timestamp is None or timestamp < min_timestamp:
                min_timestamp = timestamp

        if timestamp in support_break:
            if max_timestamp is None or timestamp > max_timestamp:
                max_timestamp = timestamp

    if min_timestamp is not None:
        filtered_resist.append(min_timestamp)

    if max_timestamp is not None:
        filtered_support.append(max_timestamp)

    current_timestamp += delta

resistance_break = filtered_resist
support_break = filtered_support

print('resistance_lines:')
for timestamp in filtered_resist:
    print(timestamp)

print('support_lines:')
for timestamp in filtered_support:
    print(timestamp)

In [ ]:
resistance_break_color = ['green' for i in range(len(resistance_break))]
support_break_color = ['red' for i in range(len(support_break))]

alert_color = 'orange'

vlines = [alert_date] + resistance_break + support_break
colors = [alert_color] + resistance_break_color + support_break_color

In [ ]:

kwargs = {
    'type': 'candle',
    'volume': True,
    'hlines': dict(hlines=hlines, colors=colors_list_h, linewidths=0.1), # Гор.линии уровней поддержки
    'vlines': dict(vlines=vlines, colors=colors), # Верт.линия события алерта
    'figratio': (3, 1),
    'style': plot_tools.get_style()
}


plot_ = [

    mpf.make_addplot(df['MA_50'], color='yellow', panel=0),
    mpf.make_addplot(df['ema'], color='green', panel=0),
    mpf.make_addplot(df['apo'], color='orange', panel=1),
    mpf.make_addplot(df['macd_hist'], color='blue', panel=1),
]

mpf.plot(df, addplot=plot_, **kwargs)

In [ ]:
resistance_break = pd.to_datetime(resistance_break)
resistance_df = df[df.index.isin(resistance_break)]

support_break = pd.to_datetime(support_break)
support_df = df[df.index.isin(support_break)]

In [ ]:
# Разметка пробоя уровней поддержки и сопротивления

df['breakdown'] = 0

df.loc[df.index.isin(resistance_break), 'breakdown'] = 1
df.loc[df.index.isin(support_break), 'breakdown'] = -1

In [ ]:
df[df['breakdown'] == 1]

In [ ]:
df[df['breakdown'] == -1]

In [ ]:
from IPython.display import clear_output

kwargs = {
    'type': 'candle',
    'volume': True,
    'figratio': (3, 1),
    'style': plot_tools.get_style()
}


enter_deal = []
exit_deal = []
result = 0

df_copy = pd.DataFrame(columns=['Open', 'Low', 'High', 'Close', 'Volume'], dtype='float64')
df_copy[['Open', 'Low', 'High', 'Close']] = df_copy[['Open', 'Low', 'High', 'Close']].astype('float64')
df_copy['Volume'] = df_copy['Volume'].astype('int64')

df_copy.index.name = 'Time'


for index in range(5, len(df)):

    event=False

    row = df.iloc[index]
    time_index = df.index[index]

    prev_row = df.shift(1).loc[time_index]
    prev_row_5 = df.shift(5).loc[time_index]

    price = row['Close']

    if row['breakdown'] == 1:
        print(f'Вход в позицию: {time_index}, Цена: {price}')
        enter_deal.append((time_index, price))
        event=True
    elif row['breakdown'] == 0 and row['apo_macd'] == 0 and (len(enter_deal) != len(exit_deal)):
        exit_deal.append((time_index, price))
        deal = exit_deal[-1][1] - enter_deal[-1][1]
        result += deal
        print(f'Выход из позиции: {time_index}, Цена: {price} Прибыль: {deal}')
        event=True
    elif time_index == alert_date:
        print('Alert')
        alert_color = 'orange'

        vlines = [alert_date]
        colors = [alert_color] 
        event=True
        
    else: 
        print(time_index)
        

    row_to_add = df.iloc[index][['Open', 'Low', 'High', 'Close', 'Volume']]

    row_to_add = row_to_add.astype(float)

    df_copy.loc[time_index] = row_to_add


    mpf.plot(df_copy, **kwargs)

    clear_output(wait=True)

    if event:
        time.sleep(3)
    else: time.sleep(0.2)



print()
print('Прибыль:', result)

